In [18]:
import pymupdf

In [19]:
data = pymupdf.open("../pipeline/data/Gr12_Mathematics_Learner_Eng.pdf")

In [27]:
num_pages=data.page_count
longest_page=0
page_with_longest_text=0

for page_num in range(num_pages):
    page = data.load_page(page_num)
    text = page.get_text()
    text_length = len(text)
    if text_length > longest_page:
        longest_page = text_length
        page_with_longest_text = page_num
        

print(f"Total number of pages: {num_pages}")
print(f"Page with the longest text: {page_with_longest_text} (Length: {longest_page} characters)")        

Total number of pages: 486
Page with the longest text: 4 (Length: 4571 characters)


In [12]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [15]:
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential


endpoint = "https://westeurope.api.cognitive.microsoft.com/"
model_name = "text-embedding-3-large"
deployment = "text-embedding-3-large"

api_version = "2024-02-01"

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=endpoint,
    api_key=os.getenv("OPENAI_EMBEDDING_KEY")
)

In [23]:
response = client.embeddings.create(
    input=data.get_page_text(0),
    model=deployment
)

In [24]:
response

CreateEmbeddingResponse(data=[Embedding(embedding=[0.0006970566464588046, 0.01227627880871296, -0.015799950808286667, -0.016680868342518806, 0.008744524791836739, -0.009892143309116364, 0.03940693661570549, 0.05463307723402977, -0.011799451895058155, 0.043706461787223816, 0.015226141549646854, -0.020964231342077255, 0.00465108547359705, 0.00023967635934241116, 0.015872687101364136, 0.03749962896108627, 0.008793015964329243, 0.009819407016038895, -0.000750093546230346, 0.011338788084685802, -0.01742439530789852, -0.020931903272867203, -0.013674433343112469, 0.023760540410876274, -0.004946071654558182, -0.01578378677368164, 0.03108266554772854, -0.004529858008027077, 0.02094806730747223, 0.012179297395050526, 0.011961088515818119, 0.013892642222344875, -0.0034246696159243584, -0.028932902961969376, 0.020463159307837486, 0.00574213033542037, -0.010958942584693432, 0.020010577514767647, -0.025797156617045403, 0.002460913034155965, 0.0041863806545734406, -0.01661621406674385, -0.00700289383

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SimpleField, SearchableField,
    VectorSearch, HnswAlgorithmConfiguration, VectorSearchProfile,
    SemanticSearch, SemanticConfiguration, PrioritizedFields, SemanticField
)
import os

endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
key = os.environ["AZURE_SEARCH_ADMIN_KEY"]
index_client = SearchIndexClient(endpoint, AzureKeyCredential(key))

index = SearchIndex(
    name="docs-index",
    fields=[
        SimpleField(name="id", type="Edm.String", key=True, filterable=True),
        SearchableField(name="content", type="Edm.String", analyzer_name="en.lucene"),
        SimpleField(name="source", type="Edm.String", filterable=True, facetable=True),
        SimpleField(name="content_vector", type="Collection(Edm.Single)", searchable=True,
                    vector_search_dimensions=1536, vector_search_profile_name="vec-profile")
    ],
    vector_search=VectorSearch(
        algorithms=[HnswAlgorithmConfiguration(name="hnsw")],
        profiles=[VectorSearchProfile(name="vec-profile", algorithm_configuration_name="hnsw")]
    ),
    semantic_search=SemanticSearch(
        configurations=[
            SemanticConfiguration(
                name="semantic-config",
                prioritized_fields=PrioritizedFields(
                    title_fields=[SemanticField(field_name="source")],
                    content_fields=[SemanticField(field_name="content")]
                )
            )
        ]
    )
)
index_client.create_or_update_index(index)


/Users/william/Documents/genai/.venv/lib/python3.12/site-packages/azure/search/documents/indexes/_generated/models/_models_py3.py:6005: SyntaxWarning: invalid escape sequence '\W'
  pattern: str = "\W+",
/Users/william/Documents/genai/.venv/lib/python3.12/site-packages/azure/search/documents/indexes/_generated/models/_models_py3.py:6230: SyntaxWarning: invalid escape sequence '\W'
  pattern: str = "\W+",


In [ ]:
from azure.search.documents.indexes.models import SearchIndexerDataSourceConnection, SearchIndexerDataContainer

ds = SearchIndexerDataSourceConnection(
    name="blob-ds",
    type="azureblob",
    connection_string=os.environ["BLOB_CONNECTION_STRING"],
    container=SearchIndexerDataContainer(name="my-container")
)
index_client.create_or_update_data_source_connection(ds)
